<img src ="../static/numba.png"></img>

In [85]:
from numba import cuda
import numba
import numpy as np

TILE_DIM = 32
BLOCK_ROWS = 8
MX = 2048
MY = 2048


In [86]:
@cuda.jit
def transpose(odata, idata):
    tile = cuda.shared.array((TILE_DIM, TILE_DIM), numba.types.float32)
    x = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.y
    w = cuda.gridDim.x * TILE_DIM

    if x >= MX or y >= MY:
        return  
    
    for i in range(0, TILE_DIM, BLOCK_ROWS):
        tile[cuda.threadIdx.y + i, cuda.threadIdx.x]  = idata[y + i, x]
    
    cuda.syncthreads();
    x = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.y

    for  i in range (0, TILE_DIM, BLOCK_ROWS):
        odata[y + i, x] = tile[cuda.threadIdx.x, cuda.threadIdx.y + i]


In [87]:
threads = (TILE_DIM, BLOCK_ROWS)
blocks = ((MX + TILE_DIM - 1) // TILE_DIM, (MY + TILE_DIM - 1) // TILE_DIM)

a_in = cuda.to_device(np.arange(MX*MY, dtype=np.float32).reshape(MX, MY))
a_out = cuda.device_array_like(a_in)

In [88]:
print(a_in.copy_to_host())

[[0.000000e+00 1.000000e+00 2.000000e+00 ... 2.045000e+03 2.046000e+03
  2.047000e+03]
 [2.048000e+03 2.049000e+03 2.050000e+03 ... 4.093000e+03 4.094000e+03
  4.095000e+03]
 [4.096000e+03 4.097000e+03 4.098000e+03 ... 6.141000e+03 6.142000e+03
  6.143000e+03]
 ...
 [4.188160e+06 4.188161e+06 4.188162e+06 ... 4.190205e+06 4.190206e+06
  4.190207e+06]
 [4.190208e+06 4.190209e+06 4.190210e+06 ... 4.192253e+06 4.192254e+06
  4.192255e+06]
 [4.192256e+06 4.192257e+06 4.192258e+06 ... 4.194301e+06 4.194302e+06
  4.194303e+06]]


In [89]:
%timeit transpose[blocks, threads](a_out, a_in); cuda.synchronize()

116 μs ± 1.27 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [90]:
print(a_out.copy_to_host())

[[0.000000e+00 2.048000e+03 4.096000e+03 ... 4.188160e+06 4.190208e+06
  4.192256e+06]
 [1.000000e+00 2.049000e+03 4.097000e+03 ... 4.188161e+06 4.190209e+06
  4.192257e+06]
 [2.000000e+00 2.050000e+03 4.098000e+03 ... 4.188162e+06 4.190210e+06
  4.192258e+06]
 ...
 [2.045000e+03 4.093000e+03 6.141000e+03 ... 4.190205e+06 4.192253e+06
  4.194301e+06]
 [2.046000e+03 4.094000e+03 6.142000e+03 ... 4.190206e+06 4.192254e+06
  4.194302e+06]
 [2.047000e+03 4.095000e+03 6.143000e+03 ... 4.190207e+06 4.192255e+06
  4.194303e+06]]


In [91]:
from numpy import testing

res = np.transpose(a_in.copy_to_host())
testing.assert_almost_equal(res, a_out.copy_to_host())

In [92]:
@cuda.jit
def transpose2(odata, idata):
    tile = cuda.shared.array((32, 32 + 1), numba.types.float32)
    x = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.y
    w = cuda.gridDim.x * TILE_DIM

    if x >= MX or y >= MY:
        return  
    
    for i in range(0, TILE_DIM, BLOCK_ROWS):
        tile[cuda.threadIdx.y + i, cuda.threadIdx.x]  = idata[y + i, x]
    
    cuda.syncthreads();
    x = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.y

    for  i in range (0, TILE_DIM, BLOCK_ROWS):
        odata[y + i, x] = tile[cuda.threadIdx.x, cuda.threadIdx.y + i]


In [93]:
%timeit transpose2[blocks, threads](a_out, a_in); cuda.synchronize()

87.4 μs ± 737 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
